# Seminar 1 - A Song of Graphs and Search

---

**Course**: Graphs and Network Analysis

**Degree**: Artificial Intelligence Degree (UAB)

**Topic**: Practical seminar that includes exercises from units 1 to 6

**Activity description**: Most of us are familiar with the Game of Thrones books or series. For those who do not know it, it is a fictional series from the HBO chain, inspired by the series of novels "A Song of Ice and Fire", which tells the experiences of a group of characters from different noble houses on the fictional continent of *Westeros* to have control of the Iron Throne and rule the seven kingdoms that make up the territory. The series' success has spawned many blogs and other sources about the series, with additional resources. The graphs that we propose to use in this exercise represent the characters of the series (or books) as nodes, and their co-appearance in a scene (the weights of the edges are higher if two characters appear simultaneously more times). So we have a social network of characters. We will use these graphs to work on some of the concepts seen in the first units of the course (graph and node metrics, search and routes). Finally, we will generate synthetic graphs that simulate a realistic network.

## Qualification

**Submission**: An '.ipynb' file from the colab corresponding to each group will be delivered (this very same file, adding the code blocks and explanations that correspond to each activity). To get the file you will need to go to File --> Download. Remember that you will have to answer and analyze the different problems. Coding alone will NOT be evaluated: explaining and reasoning about the solution of the problem is essential. **You should provide explanations of the obtained results for *at least* the exercises marked with the 💬 symbol**.
The outcome of this seminar will thus be an analysis of the network at different levels: global metrics, node importance, shortest paths, random graphs, and visualization.

**Delivery form**: The work must be done in **groups of two people** and delivered through the virtual campus (in the section corresponding to Seminar 1). Only one group member needs to submit.

**Doubts**: For any questions, apart from class sessions, you can contact ivan.erill@uab.cat.

**Deadline**: March 13th 23:59 CET (all day).

**Grade**: The grade of the seminars (seminar 1 + seminar 2) has a weight of 10% on the final grade of the course.


# Authors

**Lab group:** GrupLab-10

**Student 1 - X**

**Student 2 - Y **

## 1. Environment setup
----

The main libraries that will be used in this seminar are the following:

* [NetworkX](https://networkx.github.io/)
* [Pandas](https://pandas.pydata.org/)
* [Matplotlib](https://matplotlib.org/)
* [NumPy](https://numpy.org/)



In [ ]:
!pip install --upgrade scipy networkx

In [ ]:
!apt install libgraphviz-dev
!pip install pygraphviz

In [ ]:
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from collections import Counter

## 2. Data collection

---

This seminar is based on data from *Game of Thrones* and "A Song of Ice and Fire" curated by Andrew Beveridge. Data is available from two different github repositories:

* [Book to Network](https://github.com/mathbeveridge/asoiaf)
* [Script to Network](https://github.com/mathbeveridge/gameofthrones)

In each of them, there is a *data* folder with several *.csv* files that encode nodes and edges of different networks.

To download the data in the *colab* environment you can run the following command:

```
$ !wget https://raw.githubusercontent.com/mathbeveridge/repo_name/master/data/file_id-nodes.csv
$ !wget https://raw.githubusercontent.com/mathbeveridge/repo_name/master/data/file_id-edges.csv
```


where,

* **repo_name** is the name of the repository, *asoiaf* for the Books and *gameofthrones* for the Script.
* **file_id** is the ID of the file you can find with the link. This indicates the book or season number.

For example, to download the graph of the first season of the series, we would run:

```
$ !wget https://raw.githubusercontent.com/mathbeveridge/gameofthrones/master/data/got-s1-nodes.csv
$ !wget https://raw.githubusercontent.com/mathbeveridge/gameofthrones/master/data/got-s1-edges.csv
```

The downloaded files can be found in */content/file_name*.

For this activity, we will work with the graph generated from **all the _books_**


*  **Download only the two .csv files corresponding to the graph generated from all the books (asoiaf-all)**.

In [ ]:
!wget https://raw.githubusercontent.com/mathbeveridge/asoiaf/master/data/asoiaf-all-edges.csv
!wget https://raw.githubusercontent.com/mathbeveridge/asoiaf/master/data/asoiaf-all-nodes.csv

## 3. Data load

---

The function *csv_to_graph()* creates a NetworkX graph from the *.csv* files encoding edges and nodes.

In [ ]:
def csv_to_graph(file_id_nodes: str, file_id_edges: str, origin: str = 'book') \
                    -> nx.graph:
    """Return a nx.graph

    Build a graph given a csv file for nodes and edge.
    origin controls the source of the graph to adapt the node features.
    """

    if origin == 'book':
        key1, key2 = 'weight', 'book'
    elif origin == 'script':
        key1, key2 = 'Weight', 'Season'
    else:
        raise NameError('Unknown origin {}'.format(origin))

    nodes = pd.read_csv(file_id_nodes)
    edges = pd.read_csv(file_id_edges)

    if key2 not in edges:
        key2 = 'id'

    g = nx.Graph()
    for row in nodes.iterrows():
        g.add_node(row[1]['Id'], name=row[1]['Label'])

    for row in edges.iterrows():
        g.add_edge(row[1]['Source'],row[1]['Target'],
                   weight=1/row[1][key1], id=row[1][key2])

    return g

* **Create a NetworkX graph from the downloaded files using the `csv_to_graph` function.** [Optionally, you can repeat the process with the graph generated from the series]

In [ ]:
g_book = csv_to_graph('asoiaf-all-nodes.csv', 'asoiaf-all-edges.csv', origin='book')

* **Generate a first exploratory visualization of the graph.**

In [ ]:
plt.rcParams['figure.figsize'] = [12, 12]
plt.rcParams['figure.dpi'] = 100

# Choose layout
pos = nx.spring_layout(g_book, seed=42)

# Draw nodes
nx.draw_networkx_nodes(g_book, pos, node_size=100, node_color='cyan', edgecolors='black')

# Draw edges with transparency
nx.draw_networkx_edges(g_book, pos, alpha=0.5, width=1.2, edge_color='gray')

# Show the graph
plt.title("Graph Visualization", fontsize=15)
plt.show()

## 4. General graph metrics
---

Perform a general summary of the Network properties.

* **💬  Obtain the order, size and density of the graph, as well as the average degree of its nodes.**


In [ ]:
# Order
print('Order:', g_book.order())
# Size
print('Size:', g_book.size())
# Density
print('Density:', nx.density(g_book))
# Average degree of the nodes
average_degree = sum(dict(g_book.degree()).values()) / g_book.number_of_nodes()
print("Average degree:", average_degree)

💬 : The fact that the order of the graph is 796 means that there are 796 nodes in it. In our particular case, this implies that in the graph there are represented **796 characters** of Game of Thrones. <br>
<br>
The size (2823) refers to the number of **connections** (edges) in the graph. This means that in our graph there are 2823 connections between all the people. <br>
<br>
The density measures how connected the graph is. A density of 0.0089 means that only 0.89% of all possible edges in a fully connected graph are present. This indicates that our graph is very sparse or that most nodes are not directly connected to each other. <br>
<br>
Finally, the average degree is of 7.09, meaning that each node on average has **7 connections**. While the graph is sparse overall, individual nodes still have a moderate number of connections.

* **Check that it is a connected undirected graph.**

In [ ]:
# Check that the graph is connected
if not nx.is_directed(g_book):
    print("The graph is undirected.")
else:
    print("The graph is directed.")

# Check that the graph is undirected
if nx.is_connected(g_book):
    print("The graph is connected.")
else:
    print("The graph is not connected.")

* **💬 Make a small report on the metrics of the given graph (diameter, radius, average network distance, average clustering coefficient).**

In [ ]:
# Diameter
print('Diameter:', nx.diameter(g_book))
# Radius
print('Radius:', nx.radius(g_book))
# Average network distance
print('Average network distance:', nx.average_shortest_path_length(g_book))
# Clustering coefficient
print('Clustering coefficient: ', nx.average_clustering(g_book))

💬 : The diameter of a graph is the longest shortest path between any two nodes. In our case, the diameter of our graph is 9, meaning that the two most distant nodes in the graph are 9 steps apart when following the shortest possible paths. This suggests that the graph has a relatively small structure, where even the most separated nodes are not too distant. This means that even the most distant characters are connected through a maximum of 9 other characters. (*Example: A minor character from Dorne might be linked to someone from the Night’s Watch through several intermediaries*)<br>
<br>
A radius of 5 suggests that there exists at least one character who can reach any other character within 5 steps. This implies that some characters are highly central in the story and act as bridges between different groups. (*Examples of this characters could be main characters like Tyrion, Jon Snow, Daenerys, or Varys*) <br>
<br>
An average shortest path of 3.4 means that most characters are only about 3 or 4 connections apart. This confirms that we have a small-world structure (as we saw with the diameter). Minor characters can be connected with main characters such as Jon Snow or Daenerys by just 3 or 4 nodes. <br>
<br>
A clustering coefficient of 0.49 indicates that nearly half of a character’s direct connections also know each other. This suggests the presence of distinct communities within the graph, which aligns with the story, where characters from King’s Landing primarily interact within their own circle, while Daenerys’ followers form a separate, closely connected group...

## 5. Centrality metrics: Characters' importance
---


In this section, we will study the importance of the characters according to their centrality in the graph.

* **Compute the 10 most central nodes in the network taking into account the different types of centrality (degree, betweenness, closeness and eigenvector centrality). Use also PageRank to assess importance of the characters.**

  * *centrality_bar_plot()*: Given the corresponding centrality draw a bar graph.
  * 💬 Try to reason about the changes that you observe with the different types of centrality.

In [ ]:
#centrality is a dictionary generated by networkx centrality functions
#with keys=node_ids, values=centrality_values
def centrality_bar_plot(centrality, name='betweenness', n=10):

    # Extract the top N nodes and sort by centrality value
    top_n = dict(sorted(centrality.items(), key=lambda item: item[1], reverse=True)[:n])

    values = list(top_n.values()) # Nodes' names array
    label = list(top_n.keys()) # Centrality values array

    df = pd.DataFrame({'Name': label, name: values})
    ax = df.plot.bar(x='Name', y=name, color = 'skyblue', rot=90)

    # Adjust figure size for better clarity
    plt.title(f"Top {n} Nodes by {name.capitalize()} Centrality")
    plt.xlabel("Character")
    plt.ylabel("Centrality Score")
    plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = [10, 4]

degree_centrality = nx.degree_centrality(g_book) # Degree Centrality
betweenness_centrality = nx.betweenness_centrality(g_book) # Betweenness Centrality
closeness_centrality = nx.closeness_centrality(g_book) # Closeness Centrality
eigen_centrality = nx.eigenvector_centrality(g_book) # Eigenvalue Centrality

centrality_bar_plot(degree_centrality, name='degree')
centrality_bar_plot(betweenness_centrality, name='betweenness')
centrality_bar_plot(closeness_centrality, name='closeness')
centrality_bar_plot(eigen_centrality, name='eigen')

plt.rcParams['figure.figsize'] = [12, 12]

Tyrion Lannister has the highest degree, closeness, and eigenvector centrality, while Jon Snow has the highest betweenness centrality. The fact that the most central characters change depending on the metric makes sense when we look at what each measure represents. <br>

- Degree Centrality: Tyrion has a high degree centrality because he directly interacts with many other characters.<br>
- Closeness Centrality: Since he is well-connected, he can reach other characters quickly, making him highly accessible within the network.<br>
- Eigenvector Centrality: Not only does Tyrion have many connections, but his connections are also influential (e.g., Tywin, Daenerys), making him even more central.<br>
- Betweenness Centrality: Jon Snow ranks highest here because he often serves as a bridge between different character groups. This means he connects factions that might not interact otherwise, playing a key role in uniting different parts of the network. <br>


Each metric represents a different type of importance, which is why the most central characters vary depending on the measure used.




In [ ]:
# Page rank: [you can use alpha=0.85]
pagerank_centrality = nx.pagerank(g_book, alpha=0.85)  # Add PageRank for additional insight
centrality_bar_plot(pagerank_centrality, name='pagerank')  # PageRank plot

💬 : Jon Snow also has the highest PageRank, meaning he is one of the most influential characters, as he is connected to other important ones, even if he doesn’t have the most direct connections. This is because PageRank measures importance based on connections to other important nodes.

* **What is the subgraph generated by the best connected characters?**
  * Use closeness centrality to generate the graph of the 25 most central characters.

In [ ]:
#centrality is a dictionary generated by networkx centrality functions
#with keys=node_ids, values=centrality_values
def centrality_subgraph(g, centrality, name='closeness', n=25):
    # Get the top N nodes based on centrality
    top_nodes = sorted(centrality, key=centrality.get, reverse=True)[:n]

    # Create the subgraph with only these nodes and their connections
    subgraph = g.subgraph(top_nodes)

    # Draw the subgraph
    plt.figure(figsize=(10, 6))
    pos = nx.spring_layout(subgraph, seed=42)  # Layout for better visualization
    nx.draw(subgraph, pos, with_labels=True, node_size=700, node_color='cyan', edge_color='gray', font_size=8)

    # Title
    plt.title(f"Top {n} {name.capitalize()} Centrality Characters", fontsize=12)
    plt.show()
    return subgraph  # Return subgraph for further analysis

In [ ]:
g_subgraph = centrality_subgraph(g_book, nx.closeness_centrality(g_book))

* **Draw this subgraph where the nodes are of size proportional to their centrality. Highlight the most central and the least central node in the graph (for instance, use the color of the node to highlight it).**
  * Use *closeness centrality* and scale it appropriately to emphasize the importance of different nodes.

In [ ]:
# Function to visualize the subgraph
def centrality_subgraph_highlighted(g, centrality, name='closeness', n=25):
    # Get the top N nodes based on centrality
    top_nodes = sorted(centrality, key=centrality.get, reverse=True)[:n]

    # Create the subgraph with only these nodes and their connections
    subgraph = g.subgraph(top_nodes)

    # Scale node size based on centrality
    node_sizes = np.array([centrality[node] for node in subgraph.nodes()]) * 1000

    # Identify most and least central nodes
    max_centrality_node = max(subgraph.nodes, key=lambda node: centrality[node])
    min_centrality_node = min(subgraph.nodes, key=lambda node: centrality[node])

    # Node colors: highlight most and least central nodes
    node_colors = ["red" if node == max_centrality_node else
                   "blue" if node == min_centrality_node else
                   "gray" for node in subgraph.nodes()]

    # Draw the subgraph
    plt.figure(figsize=(10, 6))
    pos = nx.spring_layout(subgraph, seed=42)  # Layout for better visualization
    nx.draw(subgraph, pos, with_labels=True, node_size=node_sizes, node_color=node_colors, edge_color="gray", font_size=8)

    plt.scatter([], [], c="red", label="Most Central Node", s=200)
    plt.scatter([], [], c="blue", label="Least Central Node", s=200)
    plt.legend()

    plt.title(f"Top {n} {name.capitalize()} Centrality Subgraph", fontsize=12)
    plt.show()

    return subgraph  # Return subgraph for further analysis

# Full graph
closeness_centrality = nx.closeness_centrality(g_book)

# Subgraph
closeness_centrality_sub = nx.closeness_centrality(g_subgraph)

# Visualize
centrality_subgraph_highlighted(g_subgraph, closeness_centrality_sub, name='closeness', n=25)


* **Draw the tree that the BFS and DFS algorithm would generate to traverse the graph starting from the least central node of the network according to *closeness centrality*.**
  * Use *closeness centrality* and scale it appropriately to emphasize the importance of different nodes.
  * To get the positions of the nodes, you can use the `graphviz_layout(tree, prog='dot')` command.
  * 💬 Comment on the obtained result.


In [ ]:
# Full graph
closeness_centrality = nx.closeness_centrality(g_book)

# Find the least central node
least_central_node = min(closeness_centrality, key=closeness_centrality.get)

# BFS
bfs_tree = nx.bfs_tree(g_book, source=least_central_node)

# Get node sizes based on centrality
node_sizes = np.array([closeness_centrality.get(node, 0) for node in bfs_tree.nodes()]) * 1000

# Plot BFS tree
plt.figure(figsize=(12, 8))  # Increased figure size
pos_bfs = graphviz_layout(bfs_tree, prog='dot')  # Hierarchical layout
nx.draw(bfs_tree, pos_bfs, with_labels=True, node_size=node_sizes, node_color="lightblue",
        edge_color="gray", font_size=6, width=0.8, alpha=0.8)
plt.title(f"BFS Tree Traversal from {least_central_node}", fontsize=14)
plt.show()

# DFS
dfs_tree = nx.dfs_tree(g_book, source=least_central_node)

# Get node sizes based on centrality
node_sizes = np.array([closeness_centrality.get(node, 0) for node in dfs_tree.nodes()]) * 1000

# Plot DFS tree
plt.figure(figsize=(12, 8))  # Increased figure size
pos_dfs = graphviz_layout(dfs_tree, prog='dot')  # Hierarchical layout
nx.draw(dfs_tree, pos_dfs, with_labels=True, node_size=node_sizes, node_color="lightblue",
        edge_color="gray", font_size=6, width=0.8, alpha=0.8)
plt.title(f"DFS Tree Traversal from {least_central_node}", fontsize=14)
plt.show()

💬 : The BFS and DFS traversals reveal different structures in the Game of Thrones character network, but due to the large dataset, both visualizations end up somewhat messy and overlapping.

The **BFS tree** forms a **clear, layered hierarchy**, expanding from Gormon Tyrell in levels. This makes it easy to see how different characters are grouped and connected, reflecting the broader structure of alliances and relationships in Westeros. It visually organizes the network into well-defined clusters, showing how information or influence might spread.

In contrast, the **DFS tree** follows a **long, branching path**, diving deep into one connection before backtracking. This results in a more stretched-out structure, reflecting individual storylines that focus on personal interactions rather than overall connectivity. It captures the depth of character relationships but is less organized compared to BFS.

Both methods offer different ways to understand the network. BFS gives a **big-picture view**, while DFS emphasizes **individual paths and depth**, much like how character arcs develop in the series.

* **💬 Compute the shortest path between the least and the most central nodes in the complete graph.**

In [ ]:
# Identify the most and least central nodes
most_central_node = max(closeness_centrality, key=closeness_centrality.get)
least_central_node = min(closeness_centrality, key=closeness_centrality.get)

# Compute the shortest path
shortest_path = nx.shortest_path(g_book, source=least_central_node, target=most_central_node)

# Compute the shortest path length
shortest_path_length = nx.shortest_path_length(g_book, source=least_central_node, target=most_central_node)

# Print results
print(f"Most central node: {most_central_node}")
print(f"Least central node: {least_central_node}")
print(f"Shortest path between them: {shortest_path}")
print(f"Shortest path length: {shortest_path_length}")

💬 : In both the **full graph** and the **subgraph**, **Tyrion Lannister** stays the most central node, but the least central node changes. In the subgraph (top 25 most central nodes), it was **Benjen Stark**, but in the full graph, it turned out to be **Gormon Tyrell**. This makes sense because centrality depends on the whole network structure, and when more characters are included, the rankings shift.

The **shortest path** between them goes through six steps: Gormon-Tyrell → Walgrave → Pate-(novice) → Alleras → Aemon-Targaryen-(Maester-Aemon) → Alliser-Thorne → Tyrion-Lannister. That means Gormon is pretty far from Tyrion in terms of connections, needing several intermediaries to reach him.

A **shortest path length of 6** shows that Gormon Tyrell is in a very **disconnected part of the network**. He doesn’t have a direct line to Tyrion, which is why he has the lowest closeness centrality. Most of his connections seem to be **through more obscure or less influential characters**, like Walgrave and Pate, rather than major political figures. This means he’s not well integrated into the main hubs of the network, making it harder for him to reach key players like Tyrion quickly.

## 6. Random graph models
----
Up to this point, we have worked with a graph generated from the data extracted from the *Song of Ice and Fire* books. In the real world, however, obtaining the data needed to construct this graph can become very complex and expensive. This is one of the reasons why, over time, the synthetic generation of graphs has been studied.

In this section we will work on the different models described in class. We will generate random graphs and study their properties.

* **Generate random graphs with the Uniform, Gilbert and Barabási-Albert models. Fix the number of nodes to the order of the GoT graph. Adjust the rest of the parameters of the graph generation function to obtain graphs with similar number of edges.**

### Erdös-Rény: Uniform Model (gnm)

In [ ]:
seed = 42
g_uniform = nx.gnm_random_graph(n=796, m=2823, seed=seed)

### Erdös-Rény: Gilbert Model (gnp)


In [ ]:
# Set the number of nodes and target edges
n = 796
target_edges = 2823

# Estimate probability
p = (2 * target_edges) / (n * (n - 1))

# Generate graph
g_gilbert = nx.gnp_random_graph(n=n, p=p, seed=seed)

### Barabási-Albert Model



In [ ]:
# Set the number of nodes (n) and target edges
n = 796
target_edges = 2823

# Estimate m based on the formula
m = target_edges // (n - 1)

# Generate the graph
g_barbasi = nx.barabasi_albert_graph(n=n, m=m, seed=seed)

In [ ]:
g_dict = {'Book': g_book, 'Uniform': g_uniform, 'Erdos': g_gilbert, 'Barbasi': g_barbasi}

* **💬 Show the order and size of the graph as well as the average degree and clustering coefficient of its nodes. Compute also the intervals between the maximum and minimum centralities for each family of synthetic graphs. Make a small report of the main metrics. Which random graph resembles more closely the graph from the books?**
     * You can set the graph generation using a random seed. This way, two different runs will generate exactly the same graph.

In [ ]:
for k, g in g_dict.items():
    # Compute centralities
    degree_centrality = nx.degree_centrality(g) # Degree Centrality
    betweenness_centrality = nx.betweenness_centrality(g) # Betweenness Centrality
    closeness_centrality = nx.closeness_centrality(g) # Closeness Centrality
    eigen_centrality = nx.eigenvector_centrality(g) # Eigenvalue Centrality

    # Calculate the intervals (max - min)
    degree_interval = max(degree_centrality.values()) - min(degree_centrality.values())
    betweenness_interval = max(betweenness_centrality.values()) - min(betweenness_centrality.values())
    closeness_interval = max(closeness_centrality.values()) - min(closeness_centrality.values())
    eigen_interval = max(eigen_centrality.values()) - min(eigen_centrality.values())

    print(f"{k}: \n")
    print(f"Order = {g.order()}")
    print(f"Size = {g.size()}")
    print(f"Average degree = {sum(dict(g.degree()).values()) / g.number_of_nodes()}")
    print(f"Clustering coefficient = {nx.average_clustering(g)}")
    print(f"Degree centrality interval = {degree_interval}")
    print(f"Betweenness centrality interval = {betweenness_interval}")
    print(f"Closeness centrality interval = {closeness_interval}")
    print(f"Eigenvector centrality interval = {eigen_interval}")
    print("\n")

💬 : All synthetic graphs have the same number of nodes as the Book graph (796). However, the Barabási-Albert graph has fewer edges (2379), making it slightly smaller in terms of connections.<br>

The Barabási-Albert model has the lowest average degree (5.98), which is smaller than the Book's average degree (7.09). The Erdős–Rényi and Uniform models have average degrees around 7, closely matching the Book.<br>

The Book's graph has a high clustering coefficient (0.49), meaning its nodes form cohesive clusters. In comparison, the synthetic graphs, especially the Uniform and Erdős–Rényi models, show very low clustering (0.01 to 0.007), indicating less cohesive clusters. The Barabási-Albert model, with a clustering coefficient of 0.039, is closer to the Book's graph but still not a perfect match.<br>

The centrality intervals of the Book’s graph are relatively large, particularly for closeness (0.326) and betweenness (0.192) centrality, suggesting greater variation in node importance. Among the synthetic graphs, the Barabási-Albert model has the largest centrality intervals, especially for betweenness (0.156) and eigenvector centrality (0.316). The Uniform and Erdős–Rényi models have smaller intervals, indicating more evenly distributed centrality.<br>

Overall, the Barabási-Albert model most closely resembles the Book's graph, with higher clustering and larger centrality intervals. It also exhibits a more hierarchical structure, similar to the Book’s network. While not a perfect match, the Barabási-Albert model is the closest in terms of connectivity and centrality.




















* **💬 Check whether the networks (the three randomly generated ones and the network extracted from the books) follow a Power Law.**

In [ ]:
plt.rcParams['figure.figsize'] = [13, 5]

for k, g in g_dict.items():

    num_nodes = len(g.nodes())
    degrees = dict(g.degree())

    # Extract the list of degrees
    degree_list = list(degrees.values())

    # Create the scatter plot for node degrees
    plt.figure()
    plt.title(f"Degree Distribution of {k}")
    plt.xlabel("Node Degree", fontsize=10)
    plt.ylabel("Number of Nodes", fontsize=10)

    for degree in degree_list:
        plt.scatter(degree, degree_list.count(degree), label=f'{k} Graph', alpha=0.6)

    # Show the plot
    plt.show()

💬 : Both the network extracted from the book and the Barabási-Albert (BA) model follow a **power law distribution**, while the Uniform and Gilbert models follow a distribution closer to **normal**. <br>

The reason for this difference lies in the mechansims used to construct them. The Barabási-Albert model uses **preferential attachment**, meaning that new nodes are more likely to connect with existing nodes that already have many connections. This creates **large hubs—nodes** with many connections, leading to a power law distribution. In contrast, the Erdős–Rényi (ER) and Gilbert models treat all nodes equally, with no preference for high-degree nodes. As a result, all nodes tend to have a similar number of connections, which results in a more uniform or normal distribution. <br>

Real-world networks, like social networks, often have a few highly connected nodes (hubs) that are central to the structure of the network. This organic growth and concentration of connections is captured by the Barabási-Albert model, but not by the Uniform or Gilbert models.